In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.


In [3]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('C:\\Users\\hp pc\\Desktop\\python-project-chatbot-codes\\intents.json').read()
intents = json.loads(data_file)

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

71 documents
16 classes ['Availability', 'Creator', 'Hyperlocal', 'Revidly', 'Sankalp Mishra', 'Shikhar Suri', 'Uses', 'clans', 'company', 'competitors', 'founders', 'goodbye', 'greeting', 'name', 'options', 'thanks']
86 unique lemmatized words ["'s", ',', 'a', 'about', 'access', 'all', 'also', 'anyone', 'app', 'application', 'are', 'available', 'awesome', 'based', 'be', 'bye', 'can', 'chatting', 'clan', 'company', 'competition', 'competitor', 'could', 'created', 'creator', 'day', 'developed', 'do', 'doe', 'download', 'else', 'for', 'founded', 'founder', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'hyperlocal', 'in', 'is', 'it', 'later', 'me', 'mishra', 'more', 'name', 'networking', 'next', 'nice', 'o', 'offered', 'provide', 'revidly', 'sankalp', 'see', 'shikhar', 'situated', 'something', 'support', 'suri', 'tell', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'used', 'wa', 'we', 'what', 'where', 'which', 'who', 'you', 'your'

In [6]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

In [8]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [9]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [10]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
71/71 [==============================] - 1s 11ms/step - loss: 2.7528 - accuracy: 0.0563
Epoch 2/200
71/71 [==============================] - 0s 263us/step - loss: 2.7684 - accuracy: 0.0986
Epoch 3/200
71/71 [==============================] - 0s 256us/step - loss: 2.6923 - accuracy: 0.1127
Epoch 4/200
71/71 [==============================] - 0s 261us/step - loss: 2.6113 - accuracy: 0.1831
Epoch 5/200
71/71 [==============================] - 0s 271us/step - loss: 2.5304 - accuracy: 0.2254
Epoch 6/200
71/71 [==============================] - 0s 280us/step - loss: 2.5034 - accuracy: 0.1972
Epoch 7/200
71/71 [==============================] - 0s 277us/step - loss: 2.3043 - accuracy: 0.3239
Epoch 8/200
71/71 [==============================] - 0s 279us/step - loss: 2.2173 - accuracy: 0.3380
Epoch 9/200
71/71 [==============================] - 0s 278us/step - loss: 2.1179 - accuracy: 0.3803
Epoch 10/200
71/71 [==============================] - 0s 281us/step - loss: 2.1080 - accurac

71/71 [==============================] - 0s 261us/step - loss: 0.2357 - accuracy: 0.9296
Epoch 82/200
71/71 [==============================] - 0s 261us/step - loss: 0.1387 - accuracy: 0.9859
Epoch 83/200
71/71 [==============================] - 0s 261us/step - loss: 0.1830 - accuracy: 0.9437
Epoch 84/200
71/71 [==============================] - 0s 261us/step - loss: 0.0615 - accuracy: 0.9859
Epoch 85/200
71/71 [==============================] - 0s 247us/step - loss: 0.0711 - accuracy: 0.9859
Epoch 86/200
71/71 [==============================] - 0s 261us/step - loss: 0.1482 - accuracy: 0.9718
Epoch 87/200
71/71 [==============================] - 0s 247us/step - loss: 0.0857 - accuracy: 1.0000
Epoch 88/200
71/71 [==============================] - 0s 261us/step - loss: 0.0639 - accuracy: 1.0000
Epoch 89/200
71/71 [==============================] - 0s 261us/step - loss: 0.1657 - accuracy: 0.9577
Epoch 90/200
71/71 [==============================] - 0s 247us/step - loss: 0.0916 - accuracy: 

71/71 [==============================] - 0s 261us/step - loss: 0.0693 - accuracy: 0.9718
Epoch 161/200
71/71 [==============================] - 0s 261us/step - loss: 0.0364 - accuracy: 1.0000
Epoch 162/200
71/71 [==============================] - 0s 261us/step - loss: 0.0693 - accuracy: 0.9859
Epoch 163/200
71/71 [==============================] - 0s 261us/step - loss: 0.0503 - accuracy: 0.9859
Epoch 164/200
71/71 [==============================] - 0s 302us/step - loss: 0.0386 - accuracy: 1.0000
Epoch 165/200
71/71 [==============================] - 0s 261us/step - loss: 0.1060 - accuracy: 0.9718
Epoch 166/200
71/71 [==============================] - 0s 261us/step - loss: 0.0852 - accuracy: 0.9718
Epoch 167/200
71/71 [==============================] - 0s 289us/step - loss: 0.0593 - accuracy: 0.9859
Epoch 168/200
71/71 [==============================] - 0s 261us/step - loss: 0.0442 - accuracy: 1.0000
Epoch 169/200
71/71 [==============================] - 0s 261us/step - loss: 0.0195 - a